In [375]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

In [376]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

def V(num_qubits: int):
    '''
        Create a ansatz V with n qubits and assign its parameters
    '''
    #Assign random parameter
    circuit = ansatz.stargraph(num_qubits=num_qubits)
    num_params = circuit.num_parameters
    x0 = 2 * np.pi * np.random.random(num_params)
    circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))
    return circuit

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calculate_rho_3 (rho, unitary_matrix, kraus_operators):
    '''
        U(dagger) @ sum(K @ rho @ K(dagger)) @ U
    '''
    rho_2 = sum(K @ rho @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    rho_3 = (np.transpose(np.conjugate(unitary_matrix)) @ rho_2 @ unitary_matrix)
    return rho_3

def create_kraus_operators(unitary_matrix):
    '''
        Create a set of Kraus Operators from the input unitary matrix, using QR decomposition
    '''
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #Q: a 2^N x 2^N matrix, N is the number of qubits
    for q in Q:
        q = np.expand_dims(q, 1)
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))
    return tf.convert_to_tensor(kraus_ops)

def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """
    rho_2 = tf.linalg.sqrtm((tf.linalg.sqrtm(rho)) @ (rho))

    # Cast to a supported type
    real_part = tf.math.real(rho_2)
    imaginary_part = tf.math.imag(rho_2)

    # Check for NaNs in both real and imaginary parts
    contains_nan_real = tf.reduce_any(tf.math.is_nan(real_part))
    contains_nan_imag = tf.reduce_any(tf.math.is_nan(imaginary_part))

    contains_nan = contains_nan_real or contains_nan_imag

    if contains_nan == True:
        rho_2 = rho

    return tf.linalg.trace(
            rho_2
            @ (tf.linalg.sqrtm(sigma))
        )

#cost func to compare 2 given rhos
def cost(rho, rho_3):
    return 1-compilation_trace_fidelity(rho, rho_3) 


In [377]:
n = 3

#Create a ansatz V with N qubits
circuit = V(n)

print(circuit)

     ┌────────────┐   ┌────────────┐   
q_0: ┤ Ry(2.5132) ├─■─┤ Ry(1.4464) ├─■─
     ├───────────┬┘ │ └────────────┘ │ 
q_1: ┤ Ry(5.451) ├──■────────────────┼─
     ├───────────┴┐                  │ 
q_2: ┤ Ry(4.3569) ├──────────────────■─
     └────────────┘                    


In [378]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data

print (unitary_matrix) #2^N x 2^N

[[-0.20768636+0.j -0.47915012+0.j -0.0917808 +0.j -0.21174612+0.j
  -0.29864158+0.j -0.68899157+0.j -0.13197574+0.j -0.30447929+0.j]
 [ 0.47915012+0.j -0.20768636+0.j  0.21174612+0.j -0.0917808 +0.j
   0.68899157+0.j -0.29864158+0.j  0.30447929+0.j -0.13197574+0.j]
 [-0.19872598+0.j  0.11733762+0.j  0.44968748+0.j -0.26551767+0.j
  -0.28575705+0.j  0.16872506+0.j  0.6466259 +0.j -0.38179983+0.j]
 [ 0.11733762+0.j  0.19872598+0.j -0.26551767+0.j -0.44968748+0.j
   0.16872506+0.j  0.28575705+0.j -0.38179983+0.j -0.6466259 +0.j]
 [ 0.29864158+0.j  0.68899157+0.j  0.13197574+0.j  0.30447929+0.j
  -0.20768636+0.j -0.47915012+0.j -0.0917808 +0.j -0.21174612+0.j]
 [ 0.68899157+0.j -0.29864158+0.j  0.30447929+0.j -0.13197574+0.j
  -0.47915012+0.j  0.20768636+0.j -0.21174612+0.j  0.0917808 +0.j]
 [ 0.28575705+0.j -0.16872506+0.j -0.6466259 +0.j  0.38179983+0.j
  -0.19872598+0.j  0.11733762+0.j  0.44968748+0.j -0.26551767+0.j]
 [ 0.16872506+0.j  0.28575705+0.j -0.38179983+0.j -0.6466259 +0.j
  -

In [379]:
# Initialize a set of Kraus Operators from the given circuit
kraus_operators = create_kraus_operators(unitary_matrix=unitary_matrix)

#print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))
print(unitary_matrix @ np.transpose(np.conjugate(unitary_matrix)))


[[ 1.00000000e+00+0.j -2.68408387e-18+0.j -2.93656757e-17+0.j
   3.01531595e-17+0.j -2.45219788e-18+0.j -3.00689362e-17+0.j
  -1.25886116e-17+0.j  1.66868118e-17+0.j]
 [-2.68408387e-18+0.j  1.00000000e+00+0.j  2.09790901e-17+0.j
   2.67559302e-17+0.j -3.78897961e-17+0.j -1.47261197e-18+0.j
   3.55150452e-17+0.j -9.72380422e-18+0.j]
 [-2.93656757e-17+0.j  2.09790901e-17+0.j  1.00000000e+00+0.j
   7.61062140e-18+0.j  9.01151085e-18+0.j  1.32654723e-17+0.j
  -1.05078783e-17+0.j  4.05953636e-18+0.j]
 [ 3.01531595e-17+0.j  2.67559302e-17+0.j  7.61062140e-18+0.j
   1.00000000e+00+0.j -1.27239368e-18+0.j  1.52461926e-17+0.j
  -1.13073052e-18+0.j  7.67274148e-17+0.j]
 [-2.45219788e-18+0.j -3.78897961e-17+0.j  9.01151085e-18+0.j
  -1.27239368e-18+0.j  1.00000000e+00+0.j  4.91047975e-18+0.j
  -3.11557324e-17+0.j -3.35579370e-17+0.j]
 [-3.00689362e-17+0.j -1.47261197e-18+0.j  1.32654723e-17+0.j
   1.52461926e-17+0.j  4.91047975e-18+0.j  1.00000000e+00+0.j
  -2.46276635e-17+0.j  2.10638269e-17+0.j

In [380]:
#Initialize rho
rho = DensityMatrix.from_label('0' * n)
rho = tf.convert_to_tensor(rho)

#calculate rho_3
rho_3 = calculate_rho_3(rho=rho, kraus_operators=kraus_operators, unitary_matrix=unitary_matrix) 

#print(rho)
#print(rho3)


In [381]:
print(cost(rho=rho, rho_3=rho_3))

tf.Tensor((0.639132765733146+0j), shape=(), dtype=complex128)


In [382]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

In [383]:
#Auto Diff
'''
    rho: The initial rho
    unitary_matrix: randomly generated circle in initialization step
    kraus_operators: set of kraus operators
    n: number of qubits
    alpha: learning rate (?)'''
def calculate_derivative(rho, unitary_matrix, kraus_operators, n, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators)
    with tf.GradientTape() as tape:
        y = calculate_rho_3(rho, unitary_matrix, tensorKraus)
        f = 1 - compilation_trace_fidelity(rho, y) 
    
    # Get the gradient of y with respect to x
    c = tape.gradient(f, tensorKraus)

    # Reshape c to kN * N matrix
    c = tf.reshape(c, (2**n * 2**n,2**n))

    # Reshape kraus_operators to kN * N matrix
    kraus_operators = tf.reshape(kraus_operators, (2**n * 2**n,2**n))
    
    # Compute the projection term
    proj = c - kraus_operators @ (np.transpose(np.conjugate(c)) @ kraus_operators  + np.transpose(np.conjugate(kraus_operators)) @ c) / 2

    # Update the Kraus operators
    updated_kraus_operators = kraus_operators - alpha * proj
    
    # Return the updated Kraus operators
    return updated_kraus_operators



In [384]:
kraus_operators_copy = tf.identity(kraus_operators)

In [385]:
# try looping manually
for i in range (0, 1000):
    _cost = cost(rho, calculate_rho_3(rho, unitary_matrix, kraus_operators_copy))

    #Update Kraus Operators
    kraus_operators_copy=calculate_derivative(rho, unitary_matrix, kraus_operators_copy, n)

    #Reshape
    kraus_operators_copy = tf.reshape(kraus_operators_copy, (2**n,2**n,2**n))
    
    #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

    print (_cost)

tf.Tensor((0.639132765733146+0j), shape=(), dtype=complex128)
tf.Tensor((0.5949203095494799+0j), shape=(), dtype=complex128)
tf.Tensor((0.5526881630909489+0j), shape=(), dtype=complex128)
tf.Tensor((0.5124683457971888+0j), shape=(), dtype=complex128)
tf.Tensor((0.47428739888001115+0j), shape=(), dtype=complex128)
tf.Tensor((0.4381610080503108+0j), shape=(), dtype=complex128)
tf.Tensor((0.40409105834006376+0j), shape=(), dtype=complex128)
tf.Tensor((0.37206446250207226+0j), shape=(), dtype=complex128)
tf.Tensor((0.3420532395475543+0j), shape=(), dtype=complex128)
tf.Tensor((0.3140154435305492+0j), shape=(), dtype=complex128)
tf.Tensor((0.2878966393897153+0j), shape=(), dtype=complex128)
tf.Tensor((0.2636316963931691+0j), shape=(), dtype=complex128)
tf.Tensor((0.24114672669535453+0j), shape=(), dtype=complex128)
tf.Tensor((0.2203610414644499+0j), shape=(), dtype=complex128)
tf.Tensor((0.20118903305885572+0j), shape=(), dtype=complex128)
tf.Tensor((0.18354192072912112+0j), shape=(), dtype

In [386]:
print(sum(K @ np.transpose(np.conjugate(K)) for K in kraus_operators_copy))
print(kraus_operators_copy)

tf.Tensor(
[[ 0.75852027+0.00000000e+00j  0.00239213+9.08862582e-20j
   0.1275656 +9.17708300e-20j  0.00213769+7.37765232e-20j
   0.0596546 -1.38426845e-20j -0.09570614+9.50692165e-20j
  -0.09236784+1.04138712e-19j -0.04196105+7.27209008e-20j]
 [ 0.00239213-9.08862582e-20j  1.065898  +0.00000000e+00j
  -0.10173355-2.98688202e-19j  0.00962097-1.18860254e-19j
   0.10495548+1.62625696e-19j  0.24212849+8.21789031e-20j
   0.1243452 -1.15205875e-19j  0.09451588-9.39370352e-20j]
 [ 0.1275656 -9.17708300e-20j -0.10173355+2.98688202e-19j
   0.92271414+0.00000000e+00j -0.06332562+1.22441761e-19j
  -0.10085327+1.18715955e-19j -0.13536181+3.95413797e-19j
  -0.00339385+2.25913872e-19j -0.01341555+1.44138364e-19j]
 [ 0.00213769-7.37765232e-20j  0.00962097+1.18860254e-19j
  -0.06332562-1.22441761e-19j  0.96221614+0.00000000e+00j
   0.02666703+1.13907365e-19j  0.05733638+1.91039065e-19j
   0.04847453+4.26738303e-20j  0.03941033+1.88507840e-20j]
 [ 0.0596546 +1.38426845e-20j  0.10495548-1.62625696e-19j

In [387]:
#Check the output of U(dagger) @ sum(K @ rho @ K(dagger)) @ U - suppose to be rho
rho_out_1 = calculate_rho_3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_copy)

In [388]:
#Compare with rho
print(compilation_trace_fidelity(rho, rho_out_1))

tf.Tensor((1+1.979737065128071e-35j), shape=(), dtype=complex128)
